# 単純閾値処理

このノートブックでは、単純閾値処理について学びます。

## 目次
1. [固定閾値](#固定閾値)
2. [適応的閾値](#適応的閾値)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from skimage import filters, feature

plt.rcParams["font.family"] = "DejaVu Sans"
plt.rcParams["axes.unicode_minus"] = False
np.random.seed(42)
print("ライブラリのインポートが完了しました。")

## 固定閾値

In [ ]:
# 固定閾値処理
# 画像全体に同じ閾値を適用します

# サンプル画像の作成
img = np.zeros((200, 200), dtype=np.uint8)
img[50:150, 50:150] = 200  # 明るい領域
img = img + np.random.randint(-30, 30, (200, 200), dtype=np.int16)
img = np.clip(img, 0, 255).astype(np.uint8)

# 様々な固定閾値で二値化
_, thresh_100 = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)
_, thresh_127 = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
_, thresh_150 = cv2.threshold(img, 150, 255, cv2.THRESH_BINARY)

# 閾値処理の種類
_, thresh_binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)
_, thresh_binary_inv = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY_INV)
_, thresh_trunc = cv2.threshold(img, 127, 255, cv2.THRESH_TRUNC)
_, thresh_tozero = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO)
_, thresh_tozero_inv = cv2.threshold(img, 127, 255, cv2.THRESH_TOZERO_INV)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像')
axes[0, 0].axis('off')

axes[0, 1].imshow(thresh_100, cmap='gray')
axes[0, 1].set_title('閾値 100')
axes[0, 1].axis('off')

axes[0, 2].imshow(thresh_127, cmap='gray')
axes[0, 2].set_title('閾値 127')
axes[0, 2].axis('off')

axes[1, 0].imshow(thresh_150, cmap='gray')
axes[1, 0].set_title('閾値 150')
axes[1, 0].axis('off')

# 閾値処理の種類の比較
comparison = np.hstack([thresh_binary, thresh_binary_inv, thresh_trunc])
axes[1, 1].imshow(comparison, cmap='gray')
axes[1, 1].set_title('BINARY | BINARY_INV | TRUNC')
axes[1, 1].axis('off')

axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("固定閾値処理の種類:")
print("- THRESH_BINARY: 閾値以上を255、以下を0")
print("- THRESH_BINARY_INV: 反転")
print("- THRESH_TRUNC: 閾値以上を閾値に")
print("- THRESH_TOZERO: 閾値以下を0に")
print("- THRESH_TOZERO_INV: 反転")

## 適応的閾値

In [ ]:
# 適応的閾値処理
# 画像の局所的な特性に基づいて閾値を決定

# 照明が不均一な画像を作成
img = np.zeros((200, 200), dtype=np.uint8)
img[50:150, 50:150] = 200
# グラデーションを追加（照明の不均一をシミュレート）
for i in range(200):
    img[:, i] = np.clip(img[:, i].astype(np.int16) + i // 4, 0, 255).astype(np.uint8)

# 適応的閾値処理
# 方法1: 平均値ベース
adaptive_mean = cv2.adaptiveThreshold(
    img, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
    cv2.THRESH_BINARY, 11, 2
)

# 方法2: ガウシアン重み付き平均
adaptive_gaussian = cv2.adaptiveThreshold(
    img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    cv2.THRESH_BINARY, 11, 2
)

# 固定閾値との比較
_, thresh_fixed = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

# 異なるパラメータで比較
adaptive_mean_15 = cv2.adaptiveThreshold(
    img, 255, cv2.ADAPTIVE_THRESH_MEAN_C,
    cv2.THRESH_BINARY, 15, 2
)

# 結果の表示
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

axes[0, 0].imshow(img, cmap='gray')
axes[0, 0].set_title('元の画像（照明不均一）')
axes[0, 0].axis('off')

axes[0, 1].imshow(thresh_fixed, cmap='gray')
axes[0, 1].set_title('固定閾値（不適切）')
axes[0, 1].axis('off')

axes[0, 2].imshow(adaptive_mean, cmap='gray')
axes[0, 2].set_title('適応的閾値（平均、blockSize=11）')
axes[0, 2].axis('off')

axes[1, 0].imshow(adaptive_gaussian, cmap='gray')
axes[1, 0].set_title('適応的閾値（ガウシアン、blockSize=11）')
axes[1, 0].axis('off')

axes[1, 1].imshow(adaptive_mean_15, cmap='gray')
axes[1, 1].set_title('適応的閾値（平均、blockSize=15）')
axes[1, 1].axis('off')

axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("適応的閾値処理:")
print("- 局所的な閾値を計算")
print("- 照明が不均一な画像に有効")
print("- blockSize: 閾値計算の領域サイズ（奇数）")
print("- C: 平均値から引く定数")
print("- ADAPTIVE_THRESH_MEAN_C: 平均値ベース")
print("- ADAPTIVE_THRESH_GAUSSIAN_C: ガウシアン重み付き平均")